## Setup  

In this notebook we extend the **channel–flow** example to the classic **flow‑reversal** benchmark in density‑based topology optimisation (TO).  
For the forward Navier–Stokes–Brinkman implementation, refer to  
`solve_fluid.ipynb`.

\[
\begin{aligned}
\min_{\gamma}\;\; & 
J \;=\; -\,u_2\!\Bigl(\tfrac{L_x}{2},\,\tfrac{L_y}{2}\Bigr)
\qquad\text{(maximise \(v\)-velocity at the domain centre)} \\[6pt]
\text{s.\,t.}\;\;
& \mathbf R(\gamma,\mathbf u,p)=\mathbf 0 
    \qquad\; \text{(steady NS–Brinkman)} \\[2pt]
& g_1(\gamma)=1-\frac{\displaystyle\sum_e \gamma_e\,v_e}{V^{\ast}}\le 0
    \qquad \text{(material / porosity constraint)} \\[4pt]
& g_2(\gamma)=\Phi(\gamma)-\beta\,\Phi_{empty channel}\le 0
    \qquad \text{(dissipated‑power constraint)}
\end{aligned}
\]

with  

$$
\Phi(\gamma)=\frac{1}{2}\int_{\Omega}
\left(
\mu\bigl(\nabla\mathbf u+\nabla\mathbf u^{\mathsf T}\bigr)
:\bigl(\nabla\mathbf u+\nabla\mathbf u^{\mathsf T}\bigr)
+\alpha(\gamma)\,\mathbf u\cdot\mathbf u
\right)\,d\Omega
$$


where  

* **Design field** $\gamma_e \in [0,1]$ — element fill ratio (0 = fluid, 1 = solid)  
* **State variables** $\mathbf u$ (velocity) and $p$ (pressure)  
* **Residual** $\mathbf R(\gamma,\mathbf u,p)=\mathbf 0$ — discrete steady NS–Brinkman equations  
* $v_e$ — element area, $V^{\ast}$ — prescribed maximum solid volume  
* $\beta$ — weighting factor multiplied to dissipated power of empty channel

The objective $J$ maximises the downward jet stream, while 
$g_2$ ensures the total **dissipated power** stays below a user‑defined limit.

### Flow Reversal Boundary Conditions
To illustrate fluid TO, we consider a simple channel flow problem, as shown in the figure below:

<img src="../../figures/flow_reversal.png" alt="Channel" style="width: 500px;"/>

### Design parametrisation  

We adopt the standard **element-wise density method**: the design vector
$\gamma$ is mapped to the Brinkman damping coefficient
$\alpha(\gamma)$ via the RAMP scheme.

### Optimiser  

We employ the **Method of Moving Asymptotes (MMA)**.  
Gradients $\partial J/\partial\gamma$ and
$\partial g/\partial\gamma$ are produced automatically with JAX’s AD.

NOTE: We follow a convention similar to those found in the solid mechanics literature of Topology optimization where the solids are attributed with a pseudodensity of 1 and voids/fluids with a pseudodensity of 0. This is in contrast to literature in fluid mechanics where the vice-versa is often use. 


In [ ]:
import functools

import numpy as np
import jax.numpy as jnp
import jax
from jax.typing import ArrayLike
import matplotlib.pyplot as plt

import toflux.src.geometry as _geom
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.utils as _utils
import toflux.src.bc as _bc
import toflux.src.fe_fluid as _fea
import toflux.src.solver as _solv
import toflux.src.mma as _mma
import toflux.src.viz as _viz

jax.config.update("jax_enable_x64", True)
plt.rcParams.update(_viz.high_res_plot_settings)

_Ext = _utils.Extent
_Field = _fea.FluidField
_cmap = _viz.fluid_cmap

# Define Geometry and Mesh 

Below we build the **channel-flow** benchmark commonly used in fluid TO.

#### Geometry  
A simple channel is described in a json and passed as the geometry. This geometry is passed to the mesher.

#### Discretisation  
* The domain $(\Omega)$ is partitioned into **bilinear quadrilateral (Q1) elements**.  
* Both primary fields—velocity $(\mathbf u=(u_x,u_y))$ **and** pressure (p\)—are interpolated with the same Q1 shape functions (equal-order Q1/Q1 formulation).

#### Numerical integration  
Element contributions (mass, convection, brinkman, viscous and stability terms) are evaluated with a $(2 \times 2)$ Gauss quadrature.

This mesh serves as the baseline for the finite-element Navier–Stokes solve and for any subsequent density-based topology-optimisation runs.

In [ ]:
geom = _geom.BrepGeometry(brep_file="toflux/brep/channel_flow.json")

mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=200,
  nely_desired=40,
  dofs_per_node=3,
  gauss_order=2,
)

_viz.plot_brep(geom)

# Center Elements

Here, we select the central elements where the objective will be computed.

In [ ]:
lx, ly = mesh.bounding_box.lx, mesh.bounding_box.ly
dx, dy = mesh.elem_size[0], mesh.elem_size[1]
obj_bbox = _mesher.BoundingBox(
  x=_Ext(min=0.5 * lx - dx, max=0.5 * lx + dx),
  y=_Ext(min=0.5 * ly - dy, max=0.5 * ly + dy),
)

obj_bbox_elems = _mesher.compute_point_indices_in_box(mesh.elem_centers, obj_bbox)
obj_bbox_elem_indices = jnp.nonzero(obj_bbox_elems)[0]

In [ ]:
density = jnp.zeros(mesh.num_elems)
density = density.at[obj_bbox_elems].set(1.0)
_viz.plot_grid_mesh(mesh, density)

### Material  

For this fluid demonstration we keep the parameters dimensionless:

| Symbol | Meaning | Value |
|--------|---------|-------|
| $(\rho)$ | Mass density | **1** |
| $(\mu)$ | Dynamic viscosity | **1** |

We also prepare the *inverse permeability* field used in the Brinkman term.  
A RAMP interpolation converts the design variable $(\gamma\in[0,1])$ into the damping coefficient $(\alpha)$. In the subsequent topology optimization the brinkman penalty value damps fluid flow in solid.

In [ ]:
mat_params = _mat.FluidMaterial(
  mass_density=1.0,
  dynamic_viscosity=1.0,
)
min_mat_frac = 0.4
mat_frac = min_mat_frac * jnp.ones((mesh.num_elems,))

min_inv_permeability, max_inv_permeability = 0.0, mat_params.dynamic_viscosity * 1.0e4

inv_permeability_ext = _utils.Extent(min=min_inv_permeability, max=max_inv_permeability)

# Brinkman Penalty Visualization

We visualize the Brinkman penalty here. The initial RAMP penalty, allows fluid flow in the domain at the beginning of the optimization. As the optimization proceeds, the penalty becomes more stricter, restricting fluid flow in the solid regions.
Note: Lower inverse permeability/brinkman penalty allows fluid flow. Observe for a large value of 200 the convexity of plot indicates fluid flow trough the domain. We use this ramp penalty value in the beginning of optimization.

In [ ]:
test_mat_frac = np.linspace(0.0, 1.0)
brinkman_penalty_test = _mat.compute_ramp_interpolation(
  prop=test_mat_frac,
  ramp_penalty=200.0,
  prop_ext=inv_permeability_ext,
  mode="convex",
)
plt.figure(figsize=(8, 4))
plt.plot(test_mat_frac, brinkman_penalty_test, "-o", label="Brinkman Penalty")
plt.xlabel("Material Fraction")
plt.ylabel("Brinkman Penalty")
plt.title("Brinkman Penalty vs Material Fraction")
plt.legend()
plt.show()

## Boundary conditions



| Region | Type | Imposed values |
|--------|------|----------------|
| **Inlet (left vertical edge)** | Dirichlet | $(u = parabolic)$, $(v = 0)$ |
| **Outlet (right vertical edge)** | Dirichlet v, p | \(p = 0\), \(v = 0\) |
| **Top wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |
| **Bottom wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |

**Characteristic velocity**

The inlet velocity is chosen to give a channel Reynolds number of 1:

$$
U_c \;=\; \frac{\mathrm{Re}\,\nu}{H},
\qquad
\text{Re} = 1,\;
\quad
\nu = \frac{\mu}{\rho},
\quad
H = \text{channel height}.
$$


In [ ]:
reynolds_num = 1.0
char_velocity = reynolds_num * mat_params.kinematic_viscosity / (mesh.bounding_box.ly)

In [ ]:
face_tol = mesh.elem_size[0] * 0.5

# inlet condition
inlet_faces = _bc.identify_faces(mesh, edges=[geom.edges[3]], tol=face_tol)
n = len(inlet_faces)
x_nodes = jnp.linspace(-1.0, 1.0, n + 1)
u_nodes = char_velocity * (1.0 - x_nodes**2)
face_node_vals = jnp.stack([u_nodes[1:], u_nodes[:-1]], axis=1)
u_vel = (_Field.U_VEL, face_node_vals)
v_vel = (_Field.V_VEL, jnp.zeros_like(face_node_vals))
inlet_face_val = [u_vel, v_vel]

# outlet condition
outlet_faces = _bc.identify_faces(mesh, edges=[geom.edges[1]], tol=face_tol)
n = len(outlet_faces)
v_vel = (_Field.V_VEL, jnp.zeros(n))
pres = (_Field.PRESSURE, jnp.zeros(n))
outlet_face_val = [v_vel, pres]

# wall condition
wall_faces = _bc.identify_faces(
  mesh, edges=[geom.edges[0], geom.edges[2]], tol=face_tol
)
n = len(wall_faces)
u_vel = (_Field.U_VEL, jnp.zeros(n))
v_vel = (_Field.V_VEL, jnp.zeros(n))
wall_face_val = [u_vel, v_vel]

inlet_bc = _bc.DirichletBC(elem_faces=inlet_faces, values=inlet_face_val, name="inlet")
outlet_bc = _bc.DirichletBC(
  elem_faces=outlet_faces, values=outlet_face_val, name="outlet"
)
wall_bc = _bc.DirichletBC(elem_faces=wall_faces, values=wall_face_val, name="wall")


bc_list = [inlet_bc, outlet_bc, wall_bc]
bc = _bc.process_boundary_conditions(bc_list, mesh)

_viz.plot_bc(bc_list, mesh)

## Solver

The incompressible Navier–Stokes–Brinkman system is **intrinsically non-linear** because of the convective term .
Our solver therefore employs a *damped/modified Newton–Raphson* loop. For more details see 'solver.py'.

In [ ]:
solver_settings = {
  "linear": {
    "solver": _solv.LinearSolvers.PETSC,
    "rtol": 1.0e-5,
    "petsc_solver": {},
  },
  "nonlinear": {"max_iter": 10, "threshold": 1.0e-6},
}


# Initialize

We initialize the FEA solver with the mesh, material, boundary conditions and solver settings. We provide an inital guess of the solution with the Dirichlet condition enforced.

In [ ]:
flow_solver = _fea.FluidSolver(mesh, bc, mat_params, solver_settings=solver_settings)
init_press_vel = jnp.zeros((mesh.num_dofs,))
init_press_vel = init_press_vel.at[bc["fixed_dofs"]].set(bc["dirichlet_values"])

# Compute Dissipated Power of Empty Channel

Here we compute the dissipated power of empty channel which will be used as a constraint for optimization.

In [ ]:
brinkman_penalty_zero = jnp.zeros((mesh.num_elems,))
press_vel_empty_chnl = _solv.modified_newton_raphson_solve(
  flow_solver, init_press_vel, brinkman_penalty_zero
)
press_vel_elem_empty_chnl = press_vel_empty_chnl[mesh.elem_dof_mat]
press_elem = np.mean(press_vel_elem_empty_chnl[:, 0::3], axis=1)
u_vel_elem = np.mean(press_vel_elem_empty_chnl[:, 1::3], axis=1)
v_vel_elem = np.mean(press_vel_elem_empty_chnl[:, 2::3], axis=1)
vel_elem_mag = np.sqrt(u_vel_elem**2 + v_vel_elem**2)


elem_diss_pow = jax.vmap(flow_solver.compute_elem_dissipated_power)(
  brinkman_penalty_zero, press_vel_elem_empty_chnl, mesh.elem_node_coords
)
diss_pow_empty_chnl = jnp.sum(elem_diss_pow)


_, ax = plt.subplots(1, 1)
ax = _viz.plot_grid_mesh(mesh=mesh, field=vel_elem_mag, ax=ax, colorbar=True)
ax.set_title("Velocity Magnitude")

_, ax = plt.subplots(1, 1)
ax = _viz.plot_grid_mesh(mesh=mesh, field=press_elem, ax=ax, colorbar=True)
ax.set_title("Pressure")
plt.show()
press_elem
print("Min pressure:", press_elem.min())
print("Max pressure:", press_elem.max())
print("Dissipated power:", diss_pow_empty_chnl)

## Dissipated power coefficient


We require that the dissipated power of the optimized channel be less than $\beta$ times the dissipated power of an empty channel. We define the parameters below:

In [ ]:
diss_pow_coeff = 5.0

## Optimization

In [ ]:
@functools.partial(jax.jit, static_argnames=("flow_solver",))
def objective_function(
  mat_frac: jnp.ndarray,
  flow_solver: _fea.FluidSolver,
  ramp_penalty: float,
  init_press_vel=None,
):
  """Computes the objective function and its gradient.

  This function computes the objective as the vertical velocity of the target region in
    the flow field.

  Args:
    mat_frac: Material fraction array of shape (num_elems,). The array should contain
      values between 0 and 1, representing the material fraction at each element. Where
      0 indicates fluid and 1 indicates material. The material fraction can assume
      intermediate values between 0 and 1 during optimization.
    flow_solver: Fluid solver instance.
    ramp_penalty: Penalty factor for the Brinkman interpolation using a convex ramp
      function. The ramp penalty is  updated using a continuation scheme during
      optimization. In the beginning, the ramp penalty is set to a large value (which
      makes the mat frac vs Brinkman penalty convex ~200) to allow fluid flow through the
      entire domain. As the optimization progresses, the ramp penalty is reduced to
      allow the material fraction to converge to material or fluid.
    init_press_vel: Initial pressure-velocity field of shape (num_dofs,). This is used
      as the initial guess for the pressure-velocity field to Newton-Raphson
      iterations. It contains the Dirichlet boundary conditions applied to the
      pressure-velocity field.

  Returns:
    A tuple containing the objective value and a tuple of pressure-velocity field and
      material fraction.
  """

  def objective_wrapper(mat_frac):
    brinkman_penalty = _mat.compute_ramp_interpolation(
      prop=mat_frac,
      ramp_penalty=ramp_penalty,
      prop_ext=inv_permeability_ext,
      mode="convex",
    )
    press_vel = _solv.modified_newton_raphson_solve(
      flow_solver, init_press_vel, brinkman_penalty
    )
    press_vel_elem = press_vel[mesh.elem_dof_mat]
    v_vel_elem = jnp.mean(press_vel_elem[:, 2::3], axis=1)
    vel_monitor = jnp.mean(v_vel_elem[obj_bbox_elem_indices])

    return vel_monitor, (press_vel, mat_frac)

  (obj, (press_vel, mat_frac)), d_obj = jax.value_and_grad(
    objective_wrapper, has_aux=True
  )(mat_frac)
  return obj, d_obj.reshape((-1, 1)), press_vel, mat_frac

# Constraint

We define a simple material constraint optimization problem where we restrict the minimum material/solid the design can assume. With the user defined `min_mat_frac` we compute the material constraint. Furthermore, we impose a maximum dissipated power constraint. Once again, the gradients are computed automatically via jax's `value_and_grad`.

In [ ]:
def constraint_function(mat_frac: ArrayLike, min_mat_frac: float, ramp_penalty: float):
  """Computes the constraints for the material fraction and dissipated power.

  Args:
    mat_frac: Material fraction array of shape (num_elems,). The array should contain
      values between 0 and 1, representing the material fraction at each element. Where
      0 indicates fluid and 1 indicates material. The material fraction can assume
      intermediate values between 0 and 1 during optimization.
    min_mat_frac: Minimum material fraction threshold ensures material presence in the
        design.
   ramp_penalty: Penalty factor for the Brinkman interpolation using a convex ramp
      function. The ramp penalty is  updated using a continuation scheme during
      optimization. In the beginning, the ramp penalty is set to a large value (which
      makes the mat frac vs Brinkman penalty convex ~200) to allow fluid flow through the
      entire domain. As the optimization progresses, the ramp penalty is reduced to
      allow the material fraction to converge to material or fluid.

  Returns:
    A tuple containing the constraints and their gradients. The first element is a
    2D array of shape (num_constraints, 1) containing the constraints, and the second element is a
    2D array of shape (num_constraints, num_elems) containing the gradients of the constraints with respect to the material fraction.
  """

  def vol_constraint_wrapper(mat_frac):
    mean_mat_frac = jnp.mean(mat_frac)
    return 1.0 - (mean_mat_frac / min_mat_frac)

  def dissip_pow_constraint_wrapper(mat_frac):
    brinkman_penalty = _mat.compute_ramp_interpolation(
      prop=mat_frac,
      ramp_penalty=ramp_penalty,
      prop_ext=inv_permeability_ext,
      mode="convex",
    )
    press_vel = _solv.modified_newton_raphson_solve(
      flow_solver, init_press_vel, brinkman_penalty
    )
    press_vel_elem = press_vel[mesh.elem_dof_mat]
    elem_diss_pow = jax.vmap(flow_solver.compute_elem_dissipated_power)(
      brinkman_penalty, press_vel_elem, mesh.elem_node_coords
    )
    diss_pow = jnp.sum(elem_diss_pow)
    return diss_pow / (diss_pow_coeff * diss_pow_empty_chnl) - 1.0

  mat_cons, d_mat_cons = jax.value_and_grad(vol_constraint_wrapper)(mat_frac)
  dpow_cons, d_dpow_cons = jax.value_and_grad(dissip_pow_constraint_wrapper)(mat_frac)
  return jnp.array([mat_cons, dpow_cons]).reshape((-1, 1)), jnp.stack(
    (d_mat_cons.flatten(), d_dpow_cons.flatten())
  )

# Optimize
Here, we define the optimization loop. We use the MMA optimizer. We begin the optimization with a uniform design corresponding to the minimum allowed material/solid fraction. The optimization flowchart is illustrated below:
<img src="../../figures/flowchart_flow_reversal.png" alt="flowchart" style="width: 800px;"/>

In [ ]:
def optimize_design(
  fe: _fea.FluidSolver,
  min_mat_frac: float,
  max_iter: int,
  move_limit: float = 1e-2,
  kkt_tol: float = 1e-3,
  step_tol: float = 1e-3,
  plot_interval: int = 10,
):
  """Optimizes the design using the Method of Moving Asymptotes (MMA) optimization.

  The function initializes the design variables with the minimum material fraction and
    then iteratively updates them via the MMA approach. At each iteration, the objective
    function and its gradient are computed using JAX `value_and_grad` function based
    on the current material distribution. Additionally, a material constraint and
    dissipated power constraint are enforced to ensure the design maintains at least the
    specified minimum material fraction and dissipated power.

  Args:
    fe: Fluid solver instance containing mesh, boundary conditions, and material
      properties.
    min_mat_frac: Minimum allowed material fraction to enforce material presence in the
      design.
    max_iter: Maximum number of iterations for the MMA optimization loop.
    move_limit: Maximum allowable change in the design variable per iteration.
    kkt_tol: Tolerance for the Karush-Kuhn-Tucker (KKT) optimality criteria.
    step_tol: Tolerance for the optimization step size.
    plot_interval: Number of iterations between plot updates.

  Returns:
    mma_state: The final state of the MMA optimization, including the optimized design
      variables, iteration count, and convergence flag.
    history: A dictionary recording the history of the objective function values and
    constraint violations with keys:
      'obj'      - list of objective function values,
      'vol_cons' - list of volume constraint values.
  """
  design_var = min_mat_frac * np.ones((fe.mesh.num_elems, 1))
  num_design_var = design_var.shape[0]
  num_cons = 2
  lower_bound = np.zeros((num_design_var, 1))
  upper_bound = np.ones((num_design_var, 1))
  mma_params = _mma.MMAParams(
    max_iter=max_iter,
    kkt_tol=kkt_tol,
    step_tol=step_tol,
    move_limit=move_limit,
    num_design_var=num_design_var,
    num_cons=num_cons,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
  )
  mma_state = _mma.init_mma(design_var, mma_params)

  history = {"obj": [], "vol_cons": []}
  press_vel = jnp.zeros((fe.mesh.num_dofs,))
  press_vel = press_vel.at[fe.bc["fixed_dofs"]].set(fe.bc["dirichlet_values"])
  while not mma_state.is_converged:
    ramp_penalty = max(150.0, 200.0 - 0.5 * mma_state.epoch)
    objective, grad_obj, press_vel, mat_frac = objective_function(
      mma_state.x, fe, ramp_penalty, press_vel
    )

    press_vel = jax.lax.stop_gradient(press_vel)
    press_vel = press_vel.at[fe.bc["fixed_dofs"]].set(fe.bc["dirichlet_values"])

    constr, grad_cons = constraint_function(mma_state.x, min_mat_frac, ramp_penalty)
    mma_state = _mma.update_mma(
      mma_state, mma_params, objective, grad_obj, constr, grad_cons
    )

    status = (
      f"epoch {mma_state.epoch} J {objective:.2E} mc {constr[0, 0]:.2F} "
      f"dpc {constr[1, 0]:.2F}"
    )
    print(status)
    if mma_state.epoch % plot_interval == 0 or mma_state.epoch == 1:
      _, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(
        mesh=mesh,
        field=mma_state.x.reshape(-1),
        ax=ax,
        colorbar=False,
        cmap=_cmap,
        val_range=(0.0, 1.0),
      )
      ax.set_xticks([])
      ax.set_yticks([])
      for spine in ax.spines.values():
        spine.set_visible(False)

      press_vel_elem = press_vel[mesh.elem_dof_mat]
      press_elem = np.mean(press_vel_elem[:, 0::3], axis=1)
      u_vel_elem = np.mean(press_vel_elem[:, 1::3], axis=1)
      v_vel_elem = np.mean(press_vel_elem[:, 2::3], axis=1)
      vel_elem_mag = np.sqrt(u_vel_elem**2 + v_vel_elem**2)
      _, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(mesh=mesh, field=vel_elem_mag, ax=ax, colorbar=True)
      ax.set_title("Velocity Magnitude")

      _, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(mesh=mesh, field=press_elem, ax=ax, colorbar=True)
      ax.set_title("Pressure")
      plt.show()

  return mma_state, history

In [ ]:
mma_state, u = optimize_design(
  flow_solver,
  min_mat_frac=min_mat_frac,
  max_iter=200,
  move_limit=2.0e-2,
)